In [1]:
from data_generators import get_train_test_addition

#dimensjoner og størrelser til x og y
n_digits = 2
n_max = 3*n_digits
m = 10

#definerer størrelsen på parametermatrisene
d = 15
k = 5
p = 20

#henter treningsdata
data = get_train_test_addition(n_digits,samples_per_batch=100,n_batches_train=4)

In [2]:
from utils import onehot

x = data['x_train'][0]
X = onehot(x,m)
y = data['y_train'][0]

In [3]:
data['x_train'].shape

(4, 100, 6)

In [4]:
from layers import LinearLayer,EmbedPosition,Attention,Softmax,CrossEntropy,FeedForward

embed = EmbedPosition(n_max,m,d)
att1 = Attention(d,k)
ff1 = FeedForward(d,p)
un_embed = LinearLayer(d,m)
softmax = Softmax()
loss = CrossEntropy()

In [5]:
#vi ser at dLdW_2 tilhørende ff1 (først finner vi det lineære laget l2
#så nøkkel 'w' i params dict-en, så nøkkel 'd' for å finne den deriverte)
#nå er denne bare en nullmatrise 
ff1.l2.params['w']['d']

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.

In [6]:
#"manuelt" forward pass (tilsvarende algoritme 1)
z0 = embed.forward(X)
z1 = att1.forward(z0)
z2 = ff1.forward(z1)
z = un_embed.forward(z2)
Z = softmax.forward(z)

In [7]:
#evaluerer objektfunksjonen
L = loss.forward(Z,y)
print(L)

#finner den deriverte av objektfunksjonen mhp Z
dLdZ = loss.backward()

2.3032423004432316


In [8]:
#"manuelt" backward pass (tilsvarende algoritme 2)
dLdz = softmax.backward(dLdZ)
dLdz2 = un_embed.backward(dLdz)
dLdz1 = ff1.backward(dLdz2)
dLdz0 = att1.backward(dLdz1)
embed.backward(dLdz0)

In [9]:
#etter backward pass har dLdW_2 fått verdier
ff1.l2.params['w']['d']

array([[ 1.14545798e-03,  2.05153138e-04,  2.27119346e-04,
         1.01481902e-04,  6.65581523e-04, -4.85459160e-07,
         5.64302928e-04,  1.41550526e-03,  1.01197521e-03,
         5.78389865e-04,  2.02008649e-04,  4.28407408e-04,
         1.60946478e-04,  4.85066790e-04,  1.25111230e-04,
         4.81396176e-05,  1.09113024e-03,  3.01011946e-04,
         1.91427448e-04,  1.56405206e-04],
       [ 8.61721760e-04,  9.65537738e-05,  1.22069677e-04,
         1.84438872e-05, -6.58633499e-05,  6.67048540e-05,
         3.78157768e-04,  8.64239502e-04,  7.90517069e-04,
         4.67055547e-04,  2.05334019e-04, -1.06033628e-05,
        -1.91676396e-04,  1.12511166e-04,  1.51333684e-04,
        -3.52379945e-05,  4.14441373e-04,  2.22504794e-04,
        -1.15932338e-04, -1.40525191e-04],
       [ 5.31969466e-07, -9.69488045e-05,  1.49251374e-05,
        -2.46014496e-05,  1.38064194e-04, -1.73660889e-05,
        -8.53584393e-05,  3.71837573e-05, -6.56781133e-05,
        -7.30744678e-05, -9.7

In [10]:
from neural_network import NeuralNetwork

#vi kan samle lagene i en liste som vi bruker for å
#initialisere et nevralt nettverk der vi kan bruke forward() og backward() 
#for å oppnå det samme som vi gjorde manuelt over

layers = [embed,att1,ff1,un_embed,softmax]
nn = NeuralNetwork(layers)

In [11]:
#forward pass tilsvarende algoritme 1
Z = nn.forward(X)

#beregner loss med CrossEntropy
L = loss.forward(Z,y)
print(L)

#backward pass tilsvarende algoritme 2
dLdZ = loss.backward()
nn.backward(dLdZ)

2.3032423004432316


In [12]:
#før optimering er W_2[0,0] gitt ved
#ff1 er det tredje laget i layers-listen (derav layers[2])
W_2_pre_opt = nn.layers[2].l2.params['w']['w'].copy()
print(W_2_pre_opt[0,0])

0.18303728796256158


In [ ]:
import numpy as np

xs = data['x_train']
ys = data['y_train']


n_batches = xs.shape[0]
n_iters = 100
step_size = 0.1

#treningsløkke tilsvarende algoritme 4 (med gradient descent)
for j in range(n_iters):
    losses = []
    for i in range(n_batches):
        x = xs[i]
        y = ys[i]

        X = onehot(x,m)
        Z = nn.forward(X)

        losses.append(loss.forward(Z,y))
        dLdZ = loss.backward()
        nn.backward(dLdZ)
        nn.step_gd(step_size)
    mean_loss = np.mean(losses)
    print("Iterasjon ", str(j), " L = ",mean_loss, "")

In [15]:
import numpy as np

xs = data['x_train']
ys = data['y_train']

n_batches = xs.shape[0]
n_iters = 300
step_size = 0.01


def train():
#treningsløkke tilsvarende algoritme 4 (med gradient descent)
    for j in range(n_iters):
        losses = []
        for i in range(n_batches):
            x = xs[i]
            y = ys[i]

            X = onehot(x,m)
            Z = nn.forward(X)

            losses.append(loss.forward(Z,y))
            dLdZ = loss.backward()
            nn.backward(dLdZ)
            nn.step_adam(step_size, j + 1)
        mean_loss = np.mean(losses)
        print("Iterasjon ", str(j), " L = ",mean_loss, "")

In [16]:
train()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'onehot':[0m [1m[1mCannot determine Numba type of <class 'function'>[0m
[1m
File "..\..\..\..\AppData\Local\Temp\ipykernel_15700\4271933738.py", line 19:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m

In [ ]:
#etter trening ser vi at W_2 er gitt ved

W_2_post_opt = nn.layers[2].l2.params['w']['w'].copy()
print(W_2_post_opt[0,0])

#dersom differansen er større enn null har dette parameteret endret seg etter 
#gradient descent
print(W_2_post_opt[0,0] - W_2_pre_opt[0,0])


